In [14]:
#필요한 모듈 삽입 및 설치
#!pip install finance-datareader
#!pip install requests

### 네이버 금융에서 크롤링한 결과입니다. ETF, ETN, 콜/풋옵션은 포함하지 않았습니다

from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import FinanceDataReader as fdr

In [15]:
### 데이터 전처리 과정
# 한국거래소 상장종목 전체
df_krx = fdr.StockListing('KRX')
df_krx

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,"박종익, 김세완 (각자 대표이사)",http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도
...,...,...,...,...,...,...,...,...,...,...
6446,000547,KOSPI,흥국화재2우B,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6447,000545,KOSPI,흥국화재우,NaN,NaN,NaT,NaN,NaN,NaN,NaN
6448,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),1976-06-29,12월,이환구,http://www.heung-a.com,서울특별시
6449,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,2000-02-03,12월,"정영균, 이목운, 허철호 (각자대표)",http://www.heerim.com,서울특별시


In [16]:
# 엑셀로 옮기기
# df_krx.to_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total.xlsx',index=False)

In [17]:
# 한국거래소 상장종목 정보 파악하기
df_krx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6451 entries, 0 to 6450
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          6451 non-null   object        
 1   Market          6451 non-null   object        
 2   Name            6451 non-null   object        
 3   Sector          2448 non-null   object        
 4   Industry        2429 non-null   object        
 5   ListingDate     2448 non-null   datetime64[ns]
 6   SettleMonth     2448 non-null   object        
 7   Representative  2448 non-null   object        
 8   HomePage        2266 non-null   object        
 9   Region          2448 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 554.4+ KB


In [18]:
# 필요없는 열 정리(상장일, 설립월,  홈페이지주소, 지역)
df_krx_new = df_krx.drop(['ListingDate','SettleMonth','HomePage','Region'],axis=1)

In [19]:
# 정리한 상장종목 정보 파악하기
df_krx_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6451 entries, 0 to 6450
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          6451 non-null   object
 1   Market          6451 non-null   object
 2   Name            6451 non-null   object
 3   Sector          2448 non-null   object
 4   Industry        2429 non-null   object
 5   Representative  2448 non-null   object
dtypes: object(6)
memory usage: 352.8+ KB


In [20]:
# 공백 행 확인하고 지우기
df_krx_new.isnull()

,Symbol,Market,Name,Sector,Industry,Representative
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
6446,False,False,False,True,True,True
6447,False,False,False,True,True,True
6448,False,False,False,False,False,False
6449,False,False,False,False,False,False


In [21]:
# 결측치 값 확인 - 없는 값들은 제거해야함
df_krx_new.isnull().sum()

Symbol               0
Market               0
Name                 0
Sector            4003
Industry          4022
Representative    4003
dtype: int64

In [22]:
# 결측치 값 지우기
df_krx_new = df_krx_new.dropna()
df_krx_new

,Symbol,Market,Name,Sector,Industry,Representative
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,"박종익, 김세완 (각자 대표이사)"
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",박대현
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,"채형석, 이석주(각자 대표이사)"
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,정기로
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,김영주
...,...,...,...,...,...,...
6444,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)","박철범, 오길영"
6445,000540,KOSPI,흥국화재,보험업,손해보험,권중원
6448,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),이환구
6449,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,"정영균, 이목운, 허철호 (각자대표)"


In [23]:
# 코드 정리한 종목들 엑셀로 옮기기
# df_krx_new.to_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total_new.xlsx',index=False)

In [24]:
# 종목코드 추출하기
code_symbol = df_krx_new['Symbol']

for i in code_symbol:
    print(i)

060310
095570
006840
054620
265520
211270
027410
282330
138930
001460
001040
079160
035760
000120
011150
097950
051500
058820
023460
056000
065770
083660
000590
012030
367340
367360
016610
005830
000990
139130
001530
000210
001880
375500
068790
004840
241520
155660
069730
017940
245620
037370
278990
079190
365550
050120
383220
007700
130500
114090
900290
078930
083450
006360
001250
007070
028150
045890
078150
012630
039570
089470
294870
011200
036640
082740
335870
351340
351320
373200
003560
095340
099520
175330
950170
234080
067290
001060
096760
035900
024840
105560
024120
002380
021320
036670
009070
009440
112190
119650
044180
092220
151860
016380
046440
035600
001390
226360
111870
033180
060720
001940
122450
052900
058400
114450
025000
092230
000040
044450
030200
033780
030210
036030
058850
058860
060370
093050
003550
034220
001120
051900
032640
011070
066570
108670
037560
051910
079550
006260
010120
000680
383800
023150
065150
035420
160550
053290
181710
104200
060250
005940
338100

In [136]:
# for문을 통하여 전체 종목코드 검색

### 크롬브라우저 버전이 낮으면 webdirver가 실행이 되지 않음 1. webdriver를 최신으로 재설치 2. webdriver-manager를 설치
#!pip install webdriver-manager
#from selenium import webdriver
#from webdriver_manager import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install)
###

# 최대 크기 윈도우로 시작
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")   

driver = webdriver.Chrome(options=options,executable_path=r'C:\Users\yangh\chromedriver.exe')

# 1. 데이터 생성, 데이터프레임에 들어갈 열 선택
columnlist=['Symbol','Market','Name','Sector','Industry','Representative','Price','Market Cap','Revenue(2020/12)','Operating Income(2020/12)','Net Income(2020/12)','Volume','ROE','PER','PBR','D/E Ratio','Dividend Yield(%)','52W High|Low']

df = pd.DataFrame(columns=columnlist)


# 2. 기존 한국거래소 상장종목 + 네이버 금융 웹크롤링
for i in code_symbol: # 전체가 2428개가 있음
    for row_n in range(0,2428): # df_krx_new에 있는 값에서 가져오는 것이기 때문에 코드의 일치시켜야 함
        if df_krx_new.iat[row_n,0] == str(i):
            symbol = df_krx_new.iat[row_n,0] # n행,1열의 값 가져오기
            market = df_krx_new.iat[row_n,1] # n행,2열의 값 가져오기
            name = df_krx_new.iat[row_n,2] # n행,3열의 값 가져오기
            sector = df_krx_new.iat[row_n,3] # n행,4열의 값 가져오기
            industry = df_krx_new.iat[row_n,4] # n행,5열의 값 가져오기
            representive = df_krx_new.iat[row_n,5] # n행,6열의 값 가져오기

    url = 'https://finance.naver.com/item/main.nhn?code=' + str(i) # url은 string 자료구조로 되어 있기에 str()로 변환
    driver.get(url)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')

# 3. 데이터 수집 - 줄바꿈,탭처리 제거/ 자료가 없어 수집을 멈추는 것을 막기 위해 try/except 구문 사용 
    try:
        price = soup.select_one('#chart_area > div.rate_info > div > p.no_today').text
        price = price.replace('\n','')
    except:
        price = ''
    
    try:
        market_cap = soup.select_one('#tab_con1 > div.first > table > tbody > tr.strong > td').text
        market_cap = market_cap.replace('\n','').replace('\t','')
    except:
        market_cap = ''
    
    try:
        revenue = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(1) > td:nth-child(4)').text
        revenue = revenue.replace('\n','').replace('\t','')
    except:
        revenue = ''
    
    try:
        operating_income = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(2) > td:nth-child(4)').text
        operating_income = operating_income.replace('\n','').replace('\t','')
    except:
        operating_income = ''
    
    try:
        net_income = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(3) > td:nth-child(4)').text
        net_income = net_income.replace('\n','').replace('\t','')
    except:
        net_income = ''
    
    try:
        volume = soup.select_one('#chart_area > div.rate_info > table > tbody > tr:nth-child(1) > td:nth-child(3) > em').text
        volume = volume.replace('\n','').replace('\t','')
    except:
        volume = ''
    
    try:
        roe = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(6) > td:nth-child(4)').text
        roe = roe.replace('\n','').replace('\t','')
    except:
        roe = ''

    try:
        per = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(11) > td:nth-child(4)').text
        per = per.replace('\n','').replace('\t','')
    except:
        per = ''
    
    try:
        pbr = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(13) > td:nth-child(4)').text
        pbr = pbr.replace('\n','').replace('\t','')
    except:
        pbr = ''

    try:    
        de_ratio = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(7) > td:nth-child(4)').text
        de_ratio = de_ratio.replace('\n','').replace('\t','')
    except:
        de_ratio =''
    
    try:
        dividend_yield = soup.select_one('#tab_con1 > div:nth-child(5) > table > tbody:nth-child(3) > tr:nth-child(3) > td > em').text
        dividend_yield = dividend_yield.replace('\n','').replace('\t','')
    except:
        dividend_yield = ''
    
    try:
        week52_high_low = soup.select_one('#tab_con1 > div:nth-child(4) > table > tbody > tr:nth-child(2) > td').text
        week52_high_low = week52_high_low.replace('\n','').replace('\t','')
    except:
        week52_high_low = ''
        
# 4. 데이터 저장 및 출력
    data = [symbol,market,name,sector,industry,representive,price,market_cap,revenue,operating_income,net_income,volume,roe,per,pbr,de_ratio,dividend_yield,week52_high_low]
    df = df.append(pd.Series(data, index=df.columns), ignore_index=True)

driver.close() # 데이터 수집이 완료되면 웹드라이버를 끝내기
df

,Symbol,Market,Name,Sector,Industry,Representative,Price,Market Cap,Revenue(2020/12),Operating Income(2020/12),Net Income(2020/12),Volume,ROE,PER,PBR,D/E Ratio,Dividend Yield(%),52W High|Low
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,"박종익, 김세완 (각자 대표이사)","3,085","1,427억원",226,5,-11,"302,403",-3.19,-77.50,2.51,48.83,N/A,"3,280l2,000"
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",박대현,"5,800","2,716억원","10,171",230,-33,"612,195",-1.30,-55.11,0.74,451.43,3.62,"6,440l2,700"
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,"채형석, 이석주(각자 대표이사)","34,800","4,610억원","26,200","-2,216","-2,658","42,906",-20.57,-2.17,0.49,233.90,1.15,"38,050l16,350"
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,정기로,"14,650","2,988억원",253,-109,-463,"135,203",-24.63,-3.57,0.91,65.25,N/A,"19,700l6,630"
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,김영주,"29,650","4,293억원","5,918",463,250,"230,116",19.78,15.22,2.73,168.76,0.40,"31,750l20,800"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)","박철범, 오길영","3,925","1,507억원",504,25,39,"1,098,987",6.12,19.24,1.13,34.52,0.51,"5,850l1,335"
2425,000540,KOSPI,흥국화재,보험업,손해보험,권중원,"4,615","2,965억원","44,930",259,227,"115,228",3.16,8.12,0.25,"1,725.33",N/A,"5,740l2,105"
2426,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),이환구,258,301억원,889,-58,-364,0,-379.56,-0.83,-2.55,"-2,651.36",N/A,0l0
2427,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,"정영균, 이목운, 허철호 (각자대표)","8,750","1,218억원","2,039",91,47,"716,993",8.97,11.07,0.88,185.59,1.71,"11,500l3,535"


In [137]:
# 각 종목별 비교분석을 하기위해 데이터 구조를 변경 - object -> numeric
df.dtypes

Symbol                       object
Market                       object
Name                         object
Sector                       object
Industry                     object
Representative               object
Price                        object
Market Cap                   object
Revenue(2020/12)             object
Operating Income(2020/12)    object
Net Income(2020/12)          object
Volume                       object
ROE                          object
PER                          object
PBR                          object
D/E Ratio                    object
Dividend Yield(%)            object
52W High|Low                 object
dtype: object

In [138]:
# 엑셀로 변환 - krx_total_refined
# df.to_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total_refined.xlsx',index=False)

In [140]:
# 콤마 지우는 함수 만들기
def remove_comma(x):
    return x.replace(',','')

rm_cma_columns = ['Price','Market Cap','Revenue(2020/12)','Operating Income(2020/12)','Net Income(2020/12)','Volume','ROE','PER','D/E Ratio','Dividend Yield(%)']

for a in rm_cma_columns:
    df[str(a)] = df[str(a)].apply(remove_comma)

# 조,억 단위 삭제
def remove_m(x):
    return x.replace('조','').replace('억원','')

df['Market Cap'] = df['Market Cap'].apply(remove_m)

df

,Symbol,Market,Name,Sector,Industry,Representative,Price,Market Cap,Revenue(2020/12),Operating Income(2020/12),Net Income(2020/12),Volume,ROE,PER,PBR,D/E Ratio,Dividend Yield(%),52W High|Low
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,"박종익, 김세완 (각자 대표이사)",3085,1427,226,5,-11,302403,-3.19,-77.50,2.51,48.83,N/A,"3,280l2,000"
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",박대현,5800,2716,10171,230,-33,612195,-1.30,-55.11,0.74,451.43,3.62,"6,440l2,700"
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,"채형석, 이석주(각자 대표이사)",34800,4610,26200,-2216,-2658,42906,-20.57,-2.17,0.49,233.90,1.15,"38,050l16,350"
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,정기로,14650,2988,253,-109,-463,135203,-24.63,-3.57,0.91,65.25,N/A,"19,700l6,630"
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,김영주,29650,4293,5918,463,250,230116,19.78,15.22,2.73,168.76,0.40,"31,750l20,800"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)","박철범, 오길영",3925,1507,504,25,39,1098987,6.12,19.24,1.13,34.52,0.51,"5,850l1,335"
2425,000540,KOSPI,흥국화재,보험업,손해보험,권중원,4615,2965,44930,259,227,115228,3.16,8.12,0.25,1725.33,N/A,"5,740l2,105"
2426,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),이환구,258,301,889,-58,-364,0,-379.56,-0.83,-2.55,-2651.36,N/A,0l0
2427,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,"정영균, 이목운, 허철호 (각자대표)",8750,1218,2039,91,47,716993,8.97,11.07,0.88,185.59,1.71,"11,500l3,535"


In [142]:
# 조단위 삭제
# df.to_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total_refined_조단위삭제.xlsx',index=False)

In [182]:
df = pd.read_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total_refined_조단위삭제.xlsx')
df

,Symbol,Market,Name,Sector,Industry,Representative,Price,Market Cap,Revenue(2020/12),Operating Income(2020/12),Net Income(2020/12),Volume,ROE,PER,PBR,D/E Ratio,Dividend Yield(%),52W High|Low
0,60310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,"박종익, 김세완 (각자 대표이사)",3085,1427,226,5,-11,302403,-3.19,-77.50,2.51,48.83,NaN,"3,280l2,000"
1,95570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",박대현,5800,2716,10171,230,-33,612195,-1.30,-55.11,0.74,451.43,3.62,"6,440l2,700"
2,6840,KOSPI,AK홀딩스,기타 금융업,지주사업,"채형석, 이석주(각자 대표이사)",34800,4610,26200,-2216,-2658,42906,-20.57,-2.17,0.49,233.90,1.15,"38,050l16,350"
3,54620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,정기로,14650,2988,253,-109,-463,135203,-24.63,-3.57,0.91,65.25,NaN,"19,700l6,630"
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,김영주,29650,4293,5918,463,250,230116,19.78,15.22,2.73,168.76,0.40,"31,750l20,800"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)","박철범, 오길영",3925,1507,504,25,39,1098987,6.12,19.24,1.13,34.52,0.51,"5,850l1,335"
2425,540,KOSPI,흥국화재,보험업,손해보험,권중원,4615,2965,44930,259,227,115228,3.16,8.12,0.25,1725.33,NaN,"5,740l2,105"
2426,3280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),이환구,258,301,889,-58,-364,0,-379.56,-0.83,-2.55,-2651.36,NaN,0l0
2427,37440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,"정영균, 이목운, 허철호 (각자대표)",8750,1218,2039,91,47,716993,8.97,11.07,0.88,185.59,1.71,"11,500l3,535"


In [183]:
df.dtypes

Symbol                         int64
Market                        object
Name                          object
Sector                        object
Industry                      object
Representative                object
Price                          int64
Market Cap                     int64
Revenue(2020/12)              object
Operating Income(2020/12)     object
Net Income(2020/12)           object
Volume                         int64
ROE                           object
PER                           object
PBR                           object
D/E Ratio                    float64
Dividend Yield(%)            float64
52W High|Low                  object
dtype: object

In [187]:
# 공백 오류 및 비숫자형 오류 숫자형 열로 변경하기
to_numeric_columns = ['Revenue(2020/12)','Operating Income(2020/12)','Net Income(2020/12)','ROE','PER','PBR']

for a in to_numeric_columns:
    df[str(a)] = df[str(a)].apply(pd.to_numeric, errors = 'coerce')
df

#df['Revenue(2020/12)'].apply(pd.to_numeric, errors = 'coerce')

,Symbol,Market,Name,Sector,Industry,Representative,Price,Market Cap,Revenue(2020/12),Operating Income(2020/12),Net Income(2020/12),Volume,ROE,PER,PBR,D/E Ratio,Dividend Yield(%),52W High|Low
0,60310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,"박종익, 김세완 (각자 대표이사)",3085,1427,226.0,5.0,-11.0,302403,-3.19,-77.50,2.51,48.83,NaN,"3,280l2,000"
1,95570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",박대현,5800,2716,10171.0,230.0,-33.0,612195,-1.30,-55.11,0.74,451.43,3.62,"6,440l2,700"
2,6840,KOSPI,AK홀딩스,기타 금융업,지주사업,"채형석, 이석주(각자 대표이사)",34800,4610,26200.0,-2216.0,-2658.0,42906,-20.57,-2.17,0.49,233.90,1.15,"38,050l16,350"
3,54620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,정기로,14650,2988,253.0,-109.0,-463.0,135203,-24.63,-3.57,0.91,65.25,NaN,"19,700l6,630"
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,김영주,29650,4293,5918.0,463.0,250.0,230116,19.78,15.22,2.73,168.76,0.40,"31,750l20,800"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)","박철범, 오길영",3925,1507,504.0,25.0,39.0,1098987,6.12,19.24,1.13,34.52,0.51,"5,850l1,335"
2425,540,KOSPI,흥국화재,보험업,손해보험,권중원,4615,2965,44930.0,259.0,227.0,115228,3.16,8.12,0.25,1725.33,NaN,"5,740l2,105"
2426,3280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),이환구,258,301,889.0,-58.0,-364.0,0,-379.56,-0.83,-2.55,-2651.36,NaN,0l0
2427,37440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,"정영균, 이목운, 허철호 (각자대표)",8750,1218,2039.0,91.0,47.0,716993,8.97,11.07,0.88,185.59,1.71,"11,500l3,535"


In [189]:
df.dtypes

Symbol                         int64
Market                        object
Name                          object
Sector                        object
Industry                      object
Representative                object
Price                          int64
Market Cap                     int64
Revenue(2020/12)             float64
Operating Income(2020/12)    float64
Net Income(2020/12)          float64
Volume                         int64
ROE                          float64
PER                          float64
PBR                          float64
D/E Ratio                    float64
Dividend Yield(%)            float64
52W High|Low                  object
dtype: object

In [190]:
### 최종 정리 자료 엑셀에 저장하기 - krx_total_final
# df.to_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total_final.xlsx',index=False)

In [157]:
# df1.to_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total_refined_채우기.xlsx',index=False)

In [158]:
# df.dtypes

Symbol                        int64
Market                       object
Name                         object
Sector                       object
Industry                     object
Representative               object
Price                         int64
MarketCap                     int64
Revenue(2020/12)             object
OperatingIncome(2020/12)    float64
NetIncome(2020/12)          float64
Volume                        int64
ROE                          object
PER                          object
PBR                          object
D/ERatio                    float64
DividendYield(%)            float64
52WHigh|Low                  object
dtype: object

In [155]:
# df1=df.fillna(value = 0)
# df1

,Symbol,Market,Name,Sector,Industry,Representative,Price,MarketCap,Revenue(2020/12),OperatingIncome(2020/12),NetIncome(2020/12),Volume,ROE,PER,PBR,D/ERatio,DividendYield(%),52WHigh|Low
0,60310,KOSDAQ,3S,특수목적용기계제조업,반도체웨이퍼캐리어,"박종익,김세완(각자대표이사)",3085,1427,226,5.0,-11.0,302403,-3.19,-77.50,2.51,48.83,0.00,"3,280l2,000"
1,95570,KOSPI,AJ네트웍스,산업용기계및장비임대업,"렌탈(파렛트,OA장비,건설장비)",박대현,5800,2716,10171,230.0,-33.0,612195,-1.30,-55.11,0.74,451.43,3.62,"6,440l2,700"
2,6840,KOSPI,AK홀딩스,기타금융업,지주사업,"채형석,이석주(각자대표이사)",34800,4610,26200,-2216.0,-2658.0,42906,-20.57,-2.17,0.49,233.90,1.15,"38,050l16,350"
3,54620,KOSDAQ,APS홀딩스,기타금융업,인터넷트래픽솔루션,정기로,14650,2988,253,-109.0,-463.0,135203,-24.63,-3.57,0.91,65.25,0.00,"19,700l6,630"
4,265520,KOSDAQ,AP시스템,특수목적용기계제조업,디스플레이제조장비,김영주,29650,4293,5918,463.0,250.0,230116,19.78,15.22,2.73,168.76,0.40,"31,750l20,800"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,189980,KOSDAQ,흥국에프엔비,비알코올음료및얼음제조업,"과일음료(에이드베이스,스무디,착즙쥬스등)","박철범,오길영",3925,1507,504,25.0,39.0,1098987,6.12,19.24,1.13,34.52,0.51,"5,850l1,335"
2425,540,KOSPI,흥국화재,보험업,손해보험,권중원,4615,2965,44930,259.0,227.0,115228,3.16,8.12,0.25,1725.33,0.00,"5,740l2,105"
2426,3280,KOSPI,흥아해운,해상운송업,외항화물운송업(케미컬탱커),이환구,258,301,889,-58.0,-364.0,0,-379.56,-0.83,-2.55,-2651.36,0.00,0l0
2427,37440,KOSDAQ,희림,"건축기술,엔지니어링및관련기술서비스업",설계및감리용역,"정영균,이목운,허철호(각자대표)",8750,1218,2039,91.0,47.0,716993,8.97,11.07,0.88,185.59,1.71,"11,500l3,535"


In [143]:
# #Unable to parse string " " at position 346 - 오류 해결
# def remove_blank(x):
#     x.replace(' ','')

# df = df['Revenue(2020/12)'].apply(remove_blank)

# def remove_str(x):
#     x.replace('-','')

# # # 빈 자리에 채우기
# # df = df.fillna('NA')

# # df

,Symbol,Market,Name,Sector,Industry,Representative,Price,Market Cap,Revenue(2020/12),Operating Income(2020/12),Net Income(2020/12),Volume,ROE,PER,PBR,D/E Ratio,Dividend Yield(%),52W High|Low
0,060310,KOSDAQ,3S,특수 목적용 기계 제조업,반도체 웨이퍼 캐리어,"박종익, 김세완 (각자 대표이사)",3085,1427,226,5,-11,302403,-3.19,-77.50,2.51,48.83,N/A,"3,280l2,000"
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",박대현,5800,2716,10171,230,-33,612195,-1.30,-55.11,0.74,451.43,3.62,"6,440l2,700"
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,"채형석, 이석주(각자 대표이사)",34800,4610,26200,-2216,-2658,42906,-20.57,-2.17,0.49,233.90,1.15,"38,050l16,350"
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,정기로,14650,2988,253,-109,-463,135203,-24.63,-3.57,0.91,65.25,N/A,"19,700l6,630"
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,김영주,29650,4293,5918,463,250,230116,19.78,15.22,2.73,168.76,0.40,"31,750l20,800"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,189980,KOSDAQ,흥국에프엔비,비알코올음료 및 얼음 제조업,"과일음료(에이드베이스, 스무디, 착즙쥬스 등)","박철범, 오길영",3925,1507,504,25,39,1098987,6.12,19.24,1.13,34.52,0.51,"5,850l1,335"
2425,000540,KOSPI,흥국화재,보험업,손해보험,권중원,4615,2965,44930,259,227,115228,3.16,8.12,0.25,1725.33,N/A,"5,740l2,105"
2426,003280,KOSPI,흥아해운,해상 운송업,외항화물운송업(케미컬탱커),이환구,258,301,889,-58,-364,0,-379.56,-0.83,-2.55,-2651.36,N/A,0l0
2427,037440,KOSDAQ,희림,"건축기술, 엔지니어링 및 관련 기술 서비스업",설계 및 감리용역,"정영균, 이목운, 허철호 (각자대표)",8750,1218,2039,91,47,716993,8.97,11.07,0.88,185.59,1.71,"11,500l3,535"


In [144]:
# 공백삭제
# df.to_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total_refined_공백삭제.xlsx',index=False)

In [167]:
df.dtypes

Symbol                        int64
Market                       object
Name                         object
Sector                       object
Industry                     object
Representative               object
Price                         int64
MarketCap                     int64
Revenue(2020/12)             object
OperatingIncome(2020/12)    float64
NetIncome(2020/12)          float64
Volume                        int64
ROE                          object
PER                          object
PBR                          object
D/ERatio                    float64
DividendYield(%)            float64
52WHigh|Low                  object
dtype: object

0         226.0
1       10171.0
2       26200.0
3         253.0
4        5918.0
         ...   
2424      504.0
2425    44930.0
2426      889.0
2427     2039.0
2428     1062.0
Name: Revenue(2020/12), Length: 2429, dtype: float64

In [163]:
# to_numeric_columns = ['ROE','PER','PBR']

# for a in to_numeric_columns:
#     df[str(a)] = pd.to_numeric(df[str(a)])

# df

ValueError: Unable to parse string " " at position 69

In [87]:
# df.sort_values(by='Market Cap',ascending=True)

,Symbol,Market,Name,Sector,Industry,Representative,Price,Market Cap,Revenue(2020/12),Operating Income(2020/12),Net Income(2020/12),Volume,ROE,PER,PBR,D/E Ratio,Dividend Yield(%),52W High|Low
1410,225860,KONEX,엠앤씨생명과학,기타 화학제품 제조업,"무합성화장품, 건강기능식품",김현철,75,13,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
745,225850,KONEX,미애부,생활용품 도매업,"친환경, 무합성 발효화장품",김현철,283,14,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1228,267810,KONEX,앙츠,컴퓨터 및 통신장비 수리업,기술서비스,박창진,1510,23,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1260,900080,KOSDAQ,에스앤씨엔진그룹,기타 금융업,"오토바이, 제초기, 자동차기어",천진산,21,26,1076,148.0,55.0,44066805,1.28,9.56,0.12,3.73,NaN,53l24
629,179720,KONEX,렌딩머신,소프트웨어 개발 및 공급업,"신용평가솔루션, 온라인투자연계금융서비스",이상규,2800,28,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,207940,KOSPI,삼성바이오로직스,기초 의약물질 및 생물학적 제제 제조업,바이오의약품,임존종보 (Rim John Chongbo),845000,559094,11648,2928.0,2410.0,75659,,226.80,11.88,39.69,NaN,"948,000l624,000"
1882,35720,KOSPI,카카오,"자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업",인터넷 서비스(인터넷 광고),"여민수, 조수용",135500,601525,41568,4559.0,1734.0,3635863,2.70,219.25,5.32,60.94,0.02,"136,500l50,179"
135,35420,KOSPI,NAVER,"자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업",포털 서비스 및 온라인 광고,한성숙,372500,611881,53041,12153.0,8450.0,654812,15.21,47.97,5.85,106.11,0.11,"410,000l230,500"
193,660,KOSPI,SK하이닉스,반도체 제조업,"반도체,컴퓨터,통신기기 제조,도매","박정호, 이석희 (각자대표)",128000,931843,319004,50126.0,47589.0,4535613,9.53,18.14,1.56,37.11,0.91,"150,500l71,300"


In [88]:
# ### 최종정리된 자료 엑셀로 부르기
# df.to_excel(r'C:\Users\yangh\OneDrive\바탕 화면\대학자료\2021-1\컴퓨터언어\krx_total_final.xlsx',index=False)

In [192]:
### 시가총액 오름차,내림차순
df.sort_values(by='Market Cap',ascending=True)
df.sort_values(by='Market Cap',ascending=False)

,Symbol,Market,Name,Sector,Industry,Representative,Price,Market Cap,Revenue(2020/12),Operating Income(2020/12),Net Income(2020/12),Volume,ROE,PER,PBR,D/E Ratio,Dividend Yield(%),52W High|Low
866,5930,KOSPI,삼성전자,통신 및 방송 장비 제조업,"IMT2000 서비스용 동기식 기지국,교환국장비,데이터단말기,동영상휴대폰,핵심칩,반...","김기남, 김현석, 고동진",81000,4835524,2368070.0,359939.0,264078.0,13809157,9.98,21.09,2.06,37.07,3.70,"96,800l49,900"
193,660,KOSPI,SK하이닉스,반도체 제조업,"반도체,컴퓨터,통신기기 제조,도매","박정호, 이석희 (각자대표)",128000,931843,319004.0,50126.0,47589.0,4535613,9.53,18.14,1.56,37.11,0.91,"150,500l71,300"
135,35420,KOSPI,NAVER,"자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업",포털 서비스 및 온라인 광고,한성숙,372500,611881,53041.0,12153.0,8450.0,654812,15.21,47.97,5.85,106.11,0.11,"410,000l230,500"
1882,35720,KOSPI,카카오,"자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업",인터넷 서비스(인터넷 광고),"여민수, 조수용",135500,601525,41568.0,4559.0,1734.0,3635863,2.70,219.25,5.32,60.94,0.02,"136,500l50,179"
859,207940,KOSPI,삼성바이오로직스,기초 의약물질 및 생물학적 제제 제조업,바이오의약품,임존종보 (Rim John Chongbo),845000,559094,11648.0,2928.0,2410.0,75659,NaN,226.80,11.88,39.69,NaN,"948,000l624,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,179720,KONEX,렌딩머신,소프트웨어 개발 및 공급업,"신용평가솔루션, 온라인투자연계금융서비스",이상규,2800,28,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1260,900080,KOSDAQ,에스앤씨엔진그룹,기타 금융업,"오토바이, 제초기, 자동차기어",천진산,21,26,1076.0,148.0,55.0,44066805,1.28,9.56,0.12,3.73,NaN,53l24
1228,267810,KONEX,앙츠,컴퓨터 및 통신장비 수리업,기술서비스,박창진,1510,23,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
745,225850,KONEX,미애부,생활용품 도매업,"친환경, 무합성 발효화장품",김현철,283,14,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
### PER 오름차순,내림차순
df.sort_values(by='PER',ascending=True)
df.sort_values(by='PER',ascending=False)

,Symbol,Market,Name,Sector,Industry,Representative,Price,Market Cap,Revenue(2020/12),Operating Income(2020/12),Net Income(2020/12),Volume,ROE,PER,PBR,D/E Ratio,Dividend Yield(%),52W High|Low
2329,340440,KOSDAQ,한화플러스제1호스팩,금융 지원 서비스업,기업인수,김양주,2145,101,NaN,NaN,NaN,0,NaN,27328.15,1.07,7.33,NaN,"2,190l1,980"
2398,298050,KOSPI,효성첨단소재,화학섬유 제조업,PET타이어코드,황정모,361000,16173,23946.0,342.0,68.0,102699,0.03,8489.96,2.13,523.60,NaN,"440,500l91,400"
17,58820,KOSDAQ,CMG제약,의약품 제조업,의약품,이주형,5110,7097,645.0,18.0,2.0,7734081,0.12,3734.00,4.43,7.85,NaN,"7,100l3,905"
2012,20120,KOSPI,키다리스튜디오,"자료처리, 호스팅, 포털 및 기타 인터넷 정보매개 서비스업","만화, 소설 등 콘텐츠 제작 및 유통, 판매, 컴퓨터 프로그래밍, 시스템 통합 및 관리업",김영훈,15700,5140,455.0,46.0,1.0,112568,0.22,3238.63,6.69,77.10,NaN,"20,850l6,000"
1009,49180,KOSDAQ,셀루메드,기초 의약물질 및 생물학적 제제 제조업,"의료기기, 바이오시밀러 및 헬스케어사업","유인수, 이석환",7480,2954,915.0,24.0,18.0,347903,0.51,1999.83,7.34,76.28,NaN,"9,570l5,520"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,59270,KOSDAQ,해성티피씨,기타 금속 가공제품 제조업,기어 및 동력전달장치,이건복,19050,882,153.0,-13.0,-15.0,34438,-7.46,NaN,NaN,79.06,NaN,"41,100l18,300"
2347,319400,KOSDAQ,현대무벡스,일반 목적용 기계 제조업,"물류자동화, IT서비스",현기봉,5170,5259,1975.0,166.0,113.0,90998,14.08,NaN,NaN,82.19,NaN,"7,100l2,355"
2383,378850,KOSPI,화승알앤에이,자동차 신품 부품 제조업,자동차용 고무제품,이정두,5850,1111,NaN,NaN,NaN,43410,NaN,NaN,NaN,NaN,NaN,"9,300l5,140"
2406,284420,KONEX,휴럼,기타 식품 제조업,"건강기능식품, 유산균 관련 제품",김진석,16450,625,NaN,NaN,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#################### 아래는 오류들입니다. 의미없습니다. ##########################################

In [ ]:
#https://finance.naver.com/item/main.nhn?code=278990 같은 회사는 오류가 남.AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# def get_content(driver):
#     # 1. 가격, 시가총액 데이터 추출
#     price = soup.select_one('div > p.no_today')
#     market_cap = soup.select_one('#tab_con1 > div.first > table > tbody > tr.strong > td')
    
#     # 2. 결과값 추출
#     data = [price,market_cap]
#     return data

In [ ]:
# def move_coinfo(driver):
#     coinfo = driver.find_element_by_css_selector('#content > ul > li.tab_on > a')
#     coinfo.click()
#     time.sleep(1)

In [273]:
# for row_n in range(0,10):
#     for i in code_symbol:
#         if df_krx_new.iat[row_n,0] == str(i):
#             symbol = df_krx_new.iat[row_n,0]
#             market = df_krx_new.iat[row_n,1]
#             name = df_krx_new.iat[row_n,2]
#             sector = df_krx_new.iat[row_n,3]
#             industry = df_krx_new.iat[row_n,4]
#             representive = df_krx_new.iat[row_n,5]

060310
KOSDAQ
3S
특수 목적용 기계 제조업
반도체 웨이퍼 캐리어
박종익, 김세완 (각자 대표이사)
095570
KOSPI
AJ네트웍스
산업용 기계 및 장비 임대업
렌탈(파렛트, OA장비, 건설장비)
박대현
006840
KOSPI
AK홀딩스
기타 금융업
지주사업
채형석, 이석주(각자 대표이사)
054620
KOSDAQ
APS홀딩스
기타 금융업
인터넷 트래픽 솔루션
정기로
265520
KOSDAQ
AP시스템
특수 목적용 기계 제조업
디스플레이 제조 장비
김영주
211270
KOSDAQ
AP위성
통신 및 방송 장비 제조업
위성통신 단말기
류장수
027410
KOSPI
BGF
기타 금융업
지주회사
홍정국
282330
KOSPI
BGF리테일
종합 소매업
체인화 편의점
이건준
138930
KOSPI
BNK금융지주
기타 금융업
금융지주회사
김지완
001460
KOSPI
BYC
봉제의복 제조업
메리야스,란제리 제조,도매/건축공사/부동산 임대,분양,공급
고 윤 성


In [221]:
# # for문을 통하여 전체 종목코드 검색

# ### 크롬브라우저 버전이 낮으면 webdirver가 실행이 되지 않음 1. webdriver를 최신으로 재설치 2. webdriver-manager를 설치
# #!pip install webdriver-manager
# #from selenium import webdriver
# #from webdriver_manager import ChromeDriverManager
# #driver = webdriver.Chrome(ChromeDriverManager().install)
# ###

# # 최대 크기 윈도우로 시작
# options = webdriver.ChromeOptions()
# options.add_argument("--start-maximized")   

# driver = webdriver.Chrome(options=options,executable_path=r'C:\Users\yangh\chromedriver.exe')

# columnlist=['Symbol','Market','Name','Sector','Industry','Representative','Price','Market Cap','Revenue(2020/12)','Operating Income(2020/12)','Net Income(2020/12)','Volume','ROE','PER','PBR','D/E Ratio','Dividend Yield(%)','52W High|Low']

# df = pd.DataFrame(columns=columnlist)

# for i in code_symbol[0:5]: # 전체가 2428개가 있기 때문에 10개만 추출
    
#     url = 'https://finance.naver.com/item/main.nhn?code=' + str(i) # url은 string 자료구조로 되어 있기에 str()로 변환
#     driver.get(url)
    
#     html = driver.page_source
#     soup = BeautifulSoup(html,'lxml')

#     try:
#         price = soup.select_one('#chart_area > div.rate_info > div > p.no_today').text
#         price = price.replace('\n','')
#     except:
#         price = 'None'
    
#     try:
#         market_cap = soup.select_one('#tab_con1 > div.first > table > tbody > tr.strong > td').text
#         market_cap = market_cap.replace('\n','').replace('\t','')
#     except:
#         market_cap = 'None'
    
#     try:
#         revenue = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(1) > td:nth-child(4)').text
#         revenue = revenue.replace('\n','').replace('\t','')
#     except:
#         revenue = 'None'
    
#     try:
#         operating_income = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(2) > td:nth-child(4)').text
#         operating_income = operating_income.replace('\n','').replace('\t','')
#     except:
#         operating_income = 'None'
    
#     try:
#         net_income = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(3) > td:nth-child(4)').text
#         net_income = net_income.replace('\n','').replace('\t','')
#     except:
#         net_income = 'None'
    
#     try:
#         volume = soup.select_one('#chart_area > div.rate_info > table > tbody > tr:nth-child(1) > td:nth-child(3) > em').text
#         volume = volume.replace('\n','').replace('\t','')
#     except:
#         volume = 'None'
    
#     try:
#         roe = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(6) > td:nth-child(4)').text
#         roe = roe.replace('\n','').replace('\t','')
#     except:
#         roe = 'None'

#     try:
#         per = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(11) > td:nth-child(4)').text
#         per = per.replace('\n','').replace('\t','')
#     except:
#         per = 'None'
    
#     try:
#         pbr = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(13) > td:nth-child(4)').text
#         pbr = pbr.replace('\n','').replace('\t','')
#     except:
#         pbr = 'None'

#     try:    
#         de_ratio = soup.select_one('#content > div.section.cop_analysis > div.sub_section > table > tbody > tr:nth-child(7) > td:nth-child(4)').text
#         de_ratio = de_ratio.replace('\n','').replace('\t','')
#     except:
#         de_ratio ='None'
    
#     try:
#         dividend_yield = soup.select_one('#tab_con1 > div:nth-child(5) > table > tbody:nth-child(3) > tr:nth-child(3) > td > em').text
#         dividend_yield = dividend_yield.replace('\n','').replace('\t','')
#     except:
#         dividend_yield = 'None'
    
#     try:
#         week52_high_low = soup.select_one('#tab_con1 > div:nth-child(4) > table > tbody > tr:nth-child(2) > td').text
#         week52_high_low = week52_high_low.replace('\n','').replace('\t','')
#     except:
#         week52_high_low = 'None'
    
#     data = [price,market_cap,revenue,operating_income,net_income,volume,roe,per,pbr,de_ratio,dividend_yield,week52_high_low]
#     df = df.append(pd.Series(data, index=df.columns), ignore_index=True)

# df

# #https://finance.naver.com/item/main.nhn?code=278990 같은 회사는 오류가 남.AttributeError: 'NoneType' object has no attribute 'text'

ValueError: Length of passed values is 12, index implies 18.

In [22]:
# #driver = webdriver.Chrome(options=options,executable_path=r'C:\Users\yangh\chromedriver.exe')

# #url = 'https://finance.naver.com/item/main.nhn?code=' + str(code_symbol[0])
# #driver.get(url)


# ##https://wikidocs.net/91662 - 네이버금융 실시간 주가 크롤링 

# import requests

# html = driver.page_source
# soup = BeautifulSoup(html,'lxml')

# content = soup.select('div.C4VMK > span')[0].text



IndexError: list index out of range

In [154]:
# html = driver.page_source
# soup = BeautifulSoup(html,'lxml')

# price = soup.select_one('#chart_area > div.rate_info > div > p.no_today').text
# #price= price.replace('\n','') + str('원')
# market_cap = soup.select_one('#tab_con1 > div.first > table > tbody > tr.strong > td').text
# market_cap= market_cap.replace("\t","").replace("\n","")

# columnlist=['Price','Market_cap']

# df = pd.DataFrame(columns=columnlist)
# data = [price,market_cap]
# df = df.append(pd.Series(data, index=df.columns), ignore_index=True)
# df

# #df_1 = df.append(price)
# #print(price,market_cap_replace)

# #print(price,market_cap)

# #driver.find_elements_by_css_selector('#chart_area > div.rate_info > div > p.no_today')
# #today = soup.select('#chart_area > div.rate_info > div')


,Price,Market_cap
0,"\n\n29,100\n\n","4,214억원"


In [34]:
# html
# tables = pd.read_html('https://finance.naver.com/item/main.nhn?code=060310',encoding='euc-kr')
# tables

[                  0                                    1  \
 0  전일  2,900  2,900  고가  3,1203,120  (상한가  3,7703,770  )   
 1    시가  2,9302,930       저가  2,9202,920  (하한가  2,030  )   
 
                         2  
 0   거래량  933,108  933,108  
 1  거래대금  2,837  2,837  백만  ,
      매도상위       거래량     매수상위       거개량
 0     NaN       NaN      NaN       NaN
 1    키움증권  188427.0     키움증권  198859.0
 2  미래에셋대우  151157.0     한국증권  112824.0
 3  NH투자증권  132047.0   미래에셋대우  106231.0
 4   유안타증권  101349.0   하나금융투자   85884.0
 5    한국증권   68409.0   NH투자증권   79215.0
 6  외국계추정합       0.0  +65,334   65334.0,
       날짜      종가     전일비       외국인  기관
 0    NaN     NaN     NaN       NaN NaN
 1  06/04  2950.0  상향  50  +152,227 NaN
 2  06/03  2900.0      보합    +6,725 NaN
 3  06/02  2900.0  하향  20    -14859 NaN
 4  06/01  2920.0  상향  30    +6,883 NaN
 5  05/31  2890.0  하향  15     -2747 NaN
 6  05/28  2905.0  상향  10    -23983 NaN
 7    NaN     NaN     NaN       NaN NaN,
        주요재무정보 최근 연간 실적                      

In [14]:
# #code_symbol= df_krx_new['Symbol']
# #code_symbol_list = list(code_symbol)

# driver = webdriver.Chrome(options=options,executable_path=r'C:\Users\yangh\chromedriver.exe')
# url = 'https://finance.naver.com/item/main.nhn?code=060310'
# #driver.get(url)
# new_url = url.replace('https://finance.naver.com/item/main.nhn?code=060310','https://finance.naver.com/item/main.nhn?code=095570')
# driver.get(new_url)
# time.sleep(3)


In [15]:
# # 함수 만들기(각 종목별 코드)
# def Naver_Finance_code(code_symbol):
#     url = 'https://finance.naver.com/item/main.nhn?code='+ code_symbol
#     return url

In [16]:
#code_symbol_2.iloc[0:3]
#for url in range(0,2427):
 #   url = 'https://finance.naver.com/item/main.nhn?code=' + (code_symbol_2) 
  #  print(url)

In [17]:
# from selenium import webdriver

# def Naver_Finance_code_searching(code_symbol):
#     url = 'https://finance.naver.com/item/main.nhn?code='+ code_symbol
#     return url

# #최대 크기 윈도우로 시작
# options = webdriver.ChromeOptions()
# options.add_argument("--start-maximized")   

# #네이버 접속하기
# driver = webdriver.Chrome(options=options,executable_path=r'C:\Users\yangh\chromedriver.exe')


# url = Naver_Finance_code_searching(code_symbol)

# driver.get(url)

In [18]:
# 삼성전자 코드 확인하기

# driver = webdriver.Chrome(options=options,executable_path=r'C:\Users\yangh\chromedriver.exe')
# code_symbol = '005930' #삼성전자
# url= NAVER_code(code_symbol)
# driver.get(url)


In [19]:
# code_symbol_2 = df_krx_new['Symbol']

# for i in code_symbol_2:
#     print(i)

In [20]:
# df_krx_new.info()